# 1 - Preparing data

In [35]:
import pandas as pd
from sklearn.utils import shuffle

In [43]:
books_data=pd.read_json("./data/Books_small_10000.json",lines=True)

In [44]:
books_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F2H80A1ZNN1N,B00GDM3NQC,Connie Correll,"[0, 0]","I bought both boxed sets, books 1-5. Really a...",5,Can't stop reading!,1390435200,"01 23, 2014"
1,AI3DRTKCSK4KX,B00A5MREAM,Grandma,"[0, 0]",I enjoyed this short book. But it was way way ...,3,A leaf on the wind of all hallows,1399593600,"05 9, 2014"
2,A3KAKFHY9DAC8A,0446547573,"toobusyreading ""Inspired Kathy""","[1, 1]",I love Nicholas Sparks. I&#8217;ve read everyt...,4,Great writing from Nicholas Sparks.,1404518400,"07 5, 2014"
3,ATYBCYD6BIXVL,0955809215,Chrissie,"[0, 0]",I really enjoyed this adventure and look forwa...,4,great,1389225600,"01 9, 2014"
4,A17K95SEU3J68U,0991500776,"Sirde ""artist761""","[0, 0]",It was a decent read.. typical story line. Not...,3,It was a decent read.. typical story line ...,1404864000,"07 9, 2014"


In [45]:
books_data.shape

(10000, 9)

In [46]:
def get_sentiment(data):
        if data <=2:
            return 'NEGATIVE'
        elif data == 3:
            return 'NEUTRAL'
        else:
            return 'POSITIVE'

In [47]:
books_data['sentiment'] = books_data['overall'].map(lambda x: get_sentiment(x))

In [48]:
books_data.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment
0,A1F2H80A1ZNN1N,B00GDM3NQC,Connie Correll,"[0, 0]","I bought both boxed sets, books 1-5. Really a...",5,Can't stop reading!,1390435200,"01 23, 2014",POSITIVE
1,AI3DRTKCSK4KX,B00A5MREAM,Grandma,"[0, 0]",I enjoyed this short book. But it was way way ...,3,A leaf on the wind of all hallows,1399593600,"05 9, 2014",NEUTRAL
2,A3KAKFHY9DAC8A,0446547573,"toobusyreading ""Inspired Kathy""","[1, 1]",I love Nicholas Sparks. I&#8217;ve read everyt...,4,Great writing from Nicholas Sparks.,1404518400,"07 5, 2014",POSITIVE
3,ATYBCYD6BIXVL,0955809215,Chrissie,"[0, 0]",I really enjoyed this adventure and look forwa...,4,great,1389225600,"01 9, 2014",POSITIVE
4,A17K95SEU3J68U,0991500776,"Sirde ""artist761""","[0, 0]",It was a decent read.. typical story line. Not...,3,It was a decent read.. typical story line ...,1404864000,"07 9, 2014",NEUTRAL
5,AAPAK68BFPHA1,B00HPHL9OW,Michelle Dodd,"[0, 0]",I hoped for Mia to have some peace in this boo...,5,Absolutely compelling book I felt Mia's Story ...,1389830400,"01 16, 2014",POSITIVE
6,A1ETRQ3VMHZYLK,0307451305,Kahoutek,"[0, 0]",The book has the fevered intensity of Oliver S...,2,A Fevered Mess,1394668800,"03 13, 2014",NEGATIVE
7,A3LVKEP5T8HO6D,0843960221,Beck,"[0, 0]","This is the First book in the Trilogy, and I'm...",5,The Road to a Hanging,1399075200,"05 3, 2014",POSITIVE
8,A14PRVP4JK88E7,B00JKW55EA,Rafase282,"[0, 0]",After reading the other book about introvert p...,5,The follow up!,1397606400,"04 16, 2014",POSITIVE
9,A3SOAJ7WJFHVFN,1493770691,Sara S.,"[2, 2]","I really, really like this book because the he...",5,Decency prevails!,1397779200,"04 18, 2014",POSITIVE


## Imbalanced data 

In [49]:
books_data['sentiment'].value_counts()

POSITIVE    8378
NEUTRAL      978
NEGATIVE     644
Name: sentiment, dtype: int64

In [50]:
positive_reviews = books_data[books_data['sentiment'] == 'POSITIVE']
negative_reviews = books_data[books_data['sentiment'] == 'NEGATIVE']
size_negative = negative_reviews.shape[0]
new_positive_reviews = positive_reviews.iloc[:size_negative,:]
reviews = pd.concat([negative_reviews,new_positive_reviews],axis=0)
books_data=shuffle(reviews)

In [51]:
books_data['sentiment'].value_counts()

NEGATIVE    644
POSITIVE    644
Name: sentiment, dtype: int64

# 2 - Train_test split

In [52]:
X = books_data['reviewText']
y = books_data['sentiment']

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.33,random_state=0)

In [59]:
print(X_test.head(1))
print(y_test.head(1))
print(X_train.head(1))
print(y_train.head(1))

563    The book is extremely well written. The story ...
Name: reviewText, dtype: object
563    POSITIVE
Name: sentiment, dtype: object
1964    I do not understand why this book has so many ...
Name: reviewText, dtype: object
1964    NEGATIVE
Name: sentiment, dtype: object


In [55]:
y_train.value_counts()

NEGATIVE    442
POSITIVE    420
Name: sentiment, dtype: int64

In [56]:
y_test.value_counts()

POSITIVE    224
NEGATIVE    202
Name: sentiment, dtype: int64

# 3 - Bag of Words 

### The words of the input will be the columns of the matrix and the phrases will be the rows

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
vectorizer = CountVectorizer()
X_train_vectorized=vectorizer.fit_transform(X_train)

In [65]:
X_test_vectorized=vectorizer.transform(X_test)

# 4 - Classificators

In [66]:
import numpy as np
import time
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [71]:
 dict_classifiers = {"Random Forest": RandomForestClassifier(random_state=42),
    "Nearest Neighborns": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=0),
    "Logistic Regression": LogisticRegression(solver = "liblinear",random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=0),
}

In [72]:
classifiers_names = list(dict_classifiers.keys())
classifiers_values=list(dict_classifiers.values())


In [69]:
def train_model(model):
    model.fit(X_train_vectorized,y_train)
    y_test_pred = model.predict(X_test_vectorized)
    print(classification_report(y_test, y_test_pred))
    print('\n')
    return prec,times

In [70]:
times,precs = [],[]
for key,value in zip(classifiers_names,classifiers_values):
    print('------------------{}------------------'.format(key))
    prec,Time=train_model(value)
    precs.append(prec)
    times.append(Time)

------------------Floresta Aleatória------------------
Treinou em 0.501 segundos 
Precisao =  0.831
              precision    recall  f1-score   support

    NEGATIVE       0.80      0.87      0.83       202
    POSITIVE       0.87      0.80      0.83       224

    accuracy                           0.83       426
   macro avg       0.83      0.83      0.83       426
weighted avg       0.83      0.83      0.83       426



------------------Vizinhos mais próximos------------------
Treinou em 0.001 segundos 
Precisao =  0.667
              precision    recall  f1-score   support

    NEGATIVE       0.69      0.54      0.61       202
    POSITIVE       0.65      0.78      0.71       224

    accuracy                           0.67       426
   macro avg       0.67      0.66      0.66       426
weighted avg       0.67      0.67      0.66       426



------------------Árvore de Decisão------------------
Treinou em 0.097 segundos 
Precisao =  0.655
              precision    recall  f1-s

In [73]:
model = RandomForestClassifier(random_state=0)
model.fit(X_train_vectorized,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [75]:
test = ['Very good book ','Wonderful','Waste of time','great','I really enjoyed the experience with this book, it has give me many thoughtful moments']
vetorized_test = vectorizer.transform(test)
model.predict(vetorized_test)

array(['POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE'],
      dtype=object)

# 5 - Grid_search

In [77]:
from sklearn.model_selection import GridSearchCV

In [84]:
n_estimators = [100, 300, 500, 800, 1000]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

parameters = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

grid = GridSearchCV(model, parameters, cv = 3, verbose = 2, n_jobs = 3)
grid.fit(X_train_vectorized, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  41 tasks      | elapsed:   12.5s
[Parallel(n_jobs=3)]: Done 162 tasks      | elapsed:   49.7s
[Parallel(n_jobs=3)]: Done 365 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done 648 tasks      | elapsed:  3.6min
[Parallel(n_jobs=3)]: Done 1013 tasks      | elapsed:  6.1min
[Parallel(n_jobs=3)]: Done 1458 tasks      | elapsed:  9.2min
[Parallel(n_jobs=3)]: Done 1500 out of 1500 | elapsed:  9.4min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=15,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=5,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=800, n_jobs=None,
                                              oob_score=False, random_state=0,
                                     

In [86]:
model = grid.best_estimator_
model.fit(X_train_vectorized,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [87]:
y_test_pred = model.predict(X_test_vectorized)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

    NEGATIVE       0.80      0.89      0.84       202
    POSITIVE       0.89      0.79      0.84       224

    accuracy                           0.84       426
   macro avg       0.84      0.84      0.84       426
weighted avg       0.84      0.84      0.84       426



In [92]:
test = ['Very good book ','Wonderful','Waste of time','great','I really enjoyed the experience with this book, it has give me many thoughtful moments']
vetorized_test = vectorizer.transform(test)
model.predict(vetorized_test)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE'],
      dtype=object)

# Not the best but quite good :)